## Lenet 5 con MNIST

Imports

In [8]:
import tempfile
import os
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, losses

Load dataset

In [2]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images.shape

(60000, 28, 28)

Normalize the input image so that each pixel value is between 0 to 1.

In [3]:
train_images = tf.pad(train_images, [[0, 0], [2,2], [2,2]])/255
test_images = tf.pad(test_images, [[0, 0], [2,2], [2,2]])/255
train_images.shape

TensorShape([60000, 32, 32])

Most CNN’s accept 4-dimensional tensors as inputs having the dimensions of batch size, height, width, and channel. Since MNIST images are grayscale, the last dimension does not necessarily exist. We need to expand the tensor and create a dummy dimension at axis number 3. (Recall that the tensor initially had axis 0, 1, and 2.)

In [4]:
train_images = tf.expand_dims(train_images, axis=3, name=None)
test_images = tf.expand_dims(test_images, axis=3, name=None)
train_images.shape

TensorShape([60000, 32, 32, 1])

The last 2000 samples of the training set are reserved for the validation set.

In [5]:
val_images = train_images[-2000:,:,:,:]
val_labels = train_labels[-2000:]
train_images = train_images[:-2000,:,:,:]
train_labels = train_labels[:-2000]

Define the model architecture.

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(6, 5, activation='tanh', input_shape=train_images.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(16, 5, activation='tanh'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(120, 5, activation='tanh'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 activation (Activation)     (None, 14, 14, 6)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 activation_1 (Activation)   (None, 5, 5, 16)          0

Train model

In [7]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(train_images, train_labels, batch_size=64, epochs=40, validation_data=(val_images, val_labels))

Epoch 1/40
907/907 [==============================] - 42s 44ms/step - loss: 1.4236 - accuracy: 0.4827 - val_loss: 0.2738 - val_accuracy: 0.9230
Epoch 2/40
907/907 [==============================] - 37s 40ms/step - loss: 0.3136 - accuracy: 0.9021 - val_loss: 0.1796 - val_accuracy: 0.9520
Epoch 3/40
907/907 [==============================] - 37s 40ms/step - loss: 0.2388 - accuracy: 0.9231 - val_loss: 0.1228 - val_accuracy: 0.9625
Epoch 4/40
907/907 [==============================] - 35s 39ms/step - loss: 0.1973 - accuracy: 0.9362 - val_loss: 0.1292 - val_accuracy: 0.9670
Epoch 5/40
907/907 [==============================] - 36s 40ms/step - loss: 0.1675 - accuracy: 0.9469 - val_loss: 0.1262 - val_accuracy: 0.9630
Epoch 6/40
907/907 [==============================] - 36s 40ms/step - loss: 0.1529 - accuracy: 0.9512 - val_loss: 0.1132 - val_accuracy: 0.9710
Epoch 7/40
907/907 [==============================] - 36s 40ms/step - loss: 0.1389 - accuracy: 0.9554 - val_loss: 0.1050 - val_accuracy:

Evaluate

In [20]:
start = time.time()
accuracy = model.evaluate(test_images, test_labels)
end = time.time()
print("Accuracy :",accuracy[1])
print("Images per second :", int(test_images.shape[0]/(end - start)))

313/313 [==============================] - 4s 13ms/step - loss: 0.0572 - accuracy: 0.9807
Accuracy : 0.9807000160217285
Images per second : 1880


In [23]:
# Guardar el Modelo
model.save('model.h5')
